<img align="right" width="200" height="200" src="../images/tornado3d.jpg">

# Useful Information-Rich Force Field Visualizations for Geo-Science

Goal is to create information-rich visualizations for geo-science that have meaningful interpretations for scientists and educators. 

The architecture converts [NetCDF](https://www.unidata.ucar.edu/software/netcdf/) & [GRIB](https://en.wikipedia.org/wiki/GRIB) datasets into force vector fields for rendering as [3DTextures](https://docs.unity3d.com/Manual/class-Texture3D.html) in [Unity Visual Effect Graphs](https://unity.com/visual-effect-graph). This architecture consists of the following stages:

* **Stage 1 -- Discover** - Find NetCDF or GRIB datasets that contain 3D vector fields (3D space with U-V-W values at each point) that are vital to geo-science research and education. 
* **Stage 2 -- Curate** - Prepare Store curated vector fields into library to be used by Server. 
* **Stage 3 -- Serve** - Construct REST web API to serve data to C# client within the Unity environment. 
* **Stage 4 -- Create** - Create Unity 3DTexture asset to be used in a Shader to be applied to a Material of a Gameobject. This can be accomplished either during Play dynamically or in the Editor for a static asset. 
* **Stage 5 -- Visualize** - Use 3DTesture as force field as property within VFX graph block, which has meaningful interpretations. 

Issues to be resolved:

* Mapping dataset dims to 3DTexture dims
* Specifying slices of the dataset dims to the server
* Managing a library of datasets to be visualized
* Bringing the real-world interpretation of dims into Unity viz
* Varying the VFX graph to show different aspects of the UVW field
* Rigging camera to best visualize the UVW rendering

In [ ]:
from flask import Flask, request, Response
import numpy as np

## Running Flask on Google Colab

Here are some references...
- Kshitij Pawar, Flash on Google Colab [link](https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b) - 
- Chat App Using Flask, SocketIO and Colab [link](https://colab.research.google.com/github/shanaka-desoysa/shanaka-desoysa.github.io/blob/source/content/flask/Flask_Chat_Colab.ipynb)


In [ ]:
asdfasdf

FOR LATTER - Connecting Unity with Python & a database with Flask REST webservice

Here are some references...
- Mind of Koshai, Connecting Unity... [link](http://syedakbar.co/connecting-unity-with-a-database-using-python-flask-rest-webservice/)


In [ ]:
sdfgsfg

## Flask client/server to exchange compressed numpy arrays by Andres Fernandez [ink](https://gist.github.com/andres-fr/f9c0d5993d7e7b36e838744291c26dde)

In [ ]:
################################################################################
### server.py
################################################################################

#!/usr/bin/env python
# -*- coding:utf-8 -*-


"""
Dummy server, interacts with the client receiving, altering and returning
compressed numpy arrays.

Setup/run:
 1. pip install Flask --user
 2. export FLASK_APP=server.py; flask run
"""

import io
import zlib

from flask import Flask, request, Response
import numpy as np


# ## CONFIG

SERVER_HOST= "localhost"
SERVER_PORT = 12345
API_PATH = "/api/test"

# ## HELPERS

def compress_nparr(nparr):
    """
    Returns the given numpy array as compressed bytestring,
    the uncompressed and the compressed byte size.
    """
    bytestream = io.BytesIO()
    np.save(bytestream, nparr)
    uncompressed = bytestream.getvalue()
    compressed = zlib.compress(uncompressed)
    return compressed, len(uncompressed), len(compressed)

def uncompress_nparr(bytestring):
    """
    """
    return np.load(io.BytesIO(zlib.decompress(bytestring)))



# ## MAIN SERVER DESCRIPTOR/ROUTINE

# Initialize the Flask application
app = Flask(__name__)

# route http posts to this method
@app.route(API_PATH, methods=['POST'])
def test1234():
    """
    Expects a compressed, binary np array. Decompresses it, multiplies it by 10
    and returns it compressed.
    """
    r = request
    #
    data = uncompress_nparr(r.data)
    #
    data10 = data*10
    print("\n\nReceived array (compressed size = "+\
          str(r.content_length)+"):\n"+str(data))
    resp, _, _ = compress_nparr(data10)
    return Response(response=resp, status=200,
                    mimetype="application/octet_stream")


# start flask app
app.run(host=SERVER_HOST, port=SERVER_PORT)



################################################################################
### client.py
################################################################################


#!/usr/bin/env python
# -*- coding:utf-8 -*-


"""
Dummy client, interacts with the server sending and receiving
compressed numpy arrays.

Run:
python client.py
"""


from __future__ import print_function
import io
import numpy as np
import zlib
import requests


# ## CONFIG

SERVER_HOST= "localhost"
SERVER_PORT = 12345
API_PATH = "/api/test"


# ## HELPERS

def compress_nparr(nparr):
    """
    Returns the given numpy array as compressed bytestring,
    the uncompressed and the compressed byte size.
    """
    bytestream = io.BytesIO()
    np.save(bytestream, nparr)
    uncompressed = bytestream.getvalue()
    compressed = zlib.compress(uncompressed)
    return compressed, len(uncompressed), len(compressed)

def uncompress_nparr(bytestring):
    """
    """
    return np.load(io.BytesIO(zlib.decompress(bytestring)))


# ## MAIN CLIENT ROUTINE

url = "http://"+SERVER_HOST+":"+str(SERVER_PORT)+API_PATH
while True:
    raw_input("\n\npress return...")
    arr = np.random.rand(3,3)
    compressed, u_sz, c_sz = compress_nparr(arr)
    #
    print("\nsending array to", url)
    print("size in bits (orig, compressed):", u_sz, c_sz)
    print(arr)
    #
    resp = requests.post(url, data=compressed,
                         headers={'Content-Type': 'application/octet-stream'})
    #
    print("\nresponse:")
    data = uncompress_nparr(resp.content)
    print(data)
